In [ ]:
import torch
import cv2
import tqdm
from PIL import Image
import pandas as pd
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import torch.nn.functional as F
import matplotlib.pyplot as plt
import numpy as np
import torchvision.models as models

In [ ]:
!unzip /content/drive/MyDrive/final_data_1.zip

Выходные данные были обрезаны до нескольких последних строк (5000).
  inflating: final_data/train/0/4191.jpg  
  inflating: final_data/train/0/2023.jpg  
  inflating: final_data/train/0/3508.jpg  
  inflating: final_data/train/0/1848.jpg  
  inflating: final_data/train/0/203.jpg  
  inflating: final_data/train/0/265.jpg  
  inflating: final_data/train/0/4479.jpg  
  inflating: final_data/train/0/3597.jpg  
  inflating: final_data/train/0/182.jpg  
  inflating: final_data/train/0/3237.jpg  
  inflating: final_data/train/0/4653.jpg  
  inflating: final_data/train/0/1761.jpg  
  inflating: final_data/train/0/3267.jpg  
  inflating: final_data/train/0/1407.jpg  
  inflating: final_data/train/0/756.jpg  
  inflating: final_data/train/0/3078.jpg  
  inflating: final_data/train/0/3538.jpg  
  inflating: final_data/train/0/54.jpg  
  inflating: final_data/train/0/2911.jpg  
  inflating: final_data/train/0/816.jpg  
  inflating: final_data/train/0/1300.jpg  
  inflating: final_data/train/0/1891

#Строим нейросеть#


Создаем даталоадеры для обучения нейросети

In [ ]:
images_dataset = torchvision.datasets.ImageFolder("/content/final_data/train", transform = transforms.Compose([
                                                                                transforms.ToTensor(),
                                                                                transforms.Resize(256),
                                                                                transforms.CenterCrop(224),
                                                                                # нормализуем как в ImageNet
                                                                                torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                                                                                 [0.229, 0.224, 0.225])
    ]))
images_dataloader = torch.utils.data.DataLoader(images_dataset, batch_size=16, shuffle=True)

Будем использовать предобученную на ImageNet сеть ResNet18

In [ ]:
net = models.resnet18(True, True).cuda()
net

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth


  0%|          | 0.00/44.7M [00:00<?, ?B/s]

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Заменим последний полносвязный слой сети на слой, который будет выдавать 50 значений на выходе (т.к. у нас 50 классов)

In [ ]:
net.fc = nn.Linear(512,50)

Заморозим все слои нейросети, кроме самого последнего, только что добавленного fc-слоя. Будем обучать только последний слой сети.

In [ ]:
for i, child in enumerate(net.children()):
  if i==9:
    break
  for param in child.parameters():
    param.requires_grad = False

Объявляем лосс-функцию и оптимизатор:

In [ ]:
# стандартная лосс-функция для задачи классификации
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [ ]:
# для обучения на GPU
device = 'cuda:0'
net.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

Обучаем сеть 5 эпох:

In [ ]:
n_epochs = 10
print_every = 10

total_step = len(images_dataloader)

for epoch in range(1, n_epochs+1):
  print(f'Epoch {epoch}\n')
  for batch_idx, (data, target) in enumerate(images_dataloader):
    # кладем данные на GPU
    data, target = data.to(device), target.to(device)

    # делаем шаг обучения сети
    optimizer.zero_grad()
    outputs = net(data)
    loss = criterion(outputs, target)
    loss.backward()
    optimizer.step()

    if (batch_idx) % 20 == 0:
      print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'.format(epoch, n_epochs, batch_idx, total_step, loss.item()))

Epoch 1

Epoch [1/10], Step [0/310], Loss: 0.0725
Epoch [1/10], Step [20/310], Loss: 0.0578
Epoch [1/10], Step [40/310], Loss: 0.0280
Epoch [1/10], Step [60/310], Loss: 0.0568
Epoch [1/10], Step [80/310], Loss: 0.0613
Epoch [1/10], Step [100/310], Loss: 0.0501
Epoch [1/10], Step [120/310], Loss: 0.0562
Epoch [1/10], Step [140/310], Loss: 0.0629
Epoch [1/10], Step [160/310], Loss: 0.0442
Epoch [1/10], Step [180/310], Loss: 0.0587
Epoch [1/10], Step [200/310], Loss: 0.0438
Epoch [1/10], Step [220/310], Loss: 0.0237
Epoch [1/10], Step [240/310], Loss: 0.0501
Epoch [1/10], Step [260/310], Loss: 0.1090
Epoch [1/10], Step [280/310], Loss: 0.0450
Epoch [1/10], Step [300/310], Loss: 0.0523
Epoch 2

Epoch [2/10], Step [0/310], Loss: 0.0554
Epoch [2/10], Step [20/310], Loss: 0.0594
Epoch [2/10], Step [40/310], Loss: 0.0294
Epoch [2/10], Step [60/310], Loss: 0.0622
Epoch [2/10], Step [80/310], Loss: 0.0173
Epoch [2/10], Step [100/310], Loss: 0.0453
Epoch [2/10], Step [120/310], Loss: 0.0195
Epoch

Тестируем обученную сеть на тестовом наборе картинок

In [ ]:
#image_path=f'./drive/MyDrive/ML/rucode_house/data/test/2381.jpg'
#image = cv2.imread(image_path)
transform = transforms.Compose([
                                                                                transforms.ToTensor(),
                                                                                transforms.Resize(256),
                                                                                transforms.CenterCrop(224),
                                                                                # нормализуем как в ImageNet
                                                                                torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                                                                                 [0.229, 0.224, 0.225])
])
#net.eval()
#predict = net(image)


# load your image()
img = Image.open('/content/final_data/valid/7.jpg')

# Transform
input = transform(img)

# unsqueeze batch dimension, in case you are dealing with a single image
input = input.unsqueeze(0)

input = input.to(device)

# Set model to eval
net.eval()

# Get prediction
output = net(input)

prediction = int(torch.max(output.cpu().data, 1)[1].numpy())
prediction

42

In [ ]:
result = []
transform = transforms.Compose([
                                                                                transforms.ToTensor(),
                                                                                transforms.Resize(256),
                                                                                transforms.CenterCrop(224),
                                                                                # нормализуем как в ImageNet
                                                                                torchvision.transforms.Normalize([0.485, 0.456, 0.406],
                                                                                                                 [0.229, 0.224, 0.225])
])
net.eval()
for name in range(0,250):
  image_path=f'/content/final_data/test/{name}.jpg'
  try:
    img = Image.open(image_path)
  except:
    continue
  input = transform(img)
  input = input.unsqueeze(0)
  input = input.to(device)
  output = net(input)
  prediction = int(torch.max(output.cpu().data, 1)[1].numpy()) 
  result.append(prediction)


In [ ]:
submission = pd.DataFrame(result)
submission.to_csv('ans.csv', index=False,header='label')

In [ ]:
datas = pd.read_csv("./ans.csv")
datas.head()

,0
0,7
1,26
2,44
3,11
4,15


In [ ]:
datas.info

<bound method DataFrame.info of       0
0     7
1    26
2    44
3    11
4    15
..   ..
245  15
246   3
247  13
248  22
249  36

[250 rows x 1 columns]>